In [21]:
!pip install pyngrok streamlit PyMuPDF anthropic python-dotenv

In [22]:
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("")

In [24]:
%%writefile app.py
import os
from dotenv import load_dotenv
import fitz
import streamlit as st
from anthropic import Anthropic

# Set your Claude API key
client = Anthropic(api_key="")

# Initialize flags
for key in ["generate_summary", "ask_question", "generate_glossary", "simplify_text"]:
    if key not in st.session_state:
        st.session_state[key] = False

# --------------------- Functions --------------------- #

def extract_text_from_pdf(file):
    doc = fitz.open(stream=file.read(), filetype="pdf")
    return "\n".join([page.get_text() for page in doc])

def summarize_with_claude(text):
    prompt = f"""You are a helpful academic assistant. Summarize the following paper:
    - What is it about?
    - What are the key contributions?
    - What methods were used?
    - What are the main findings?

    {text[:12000]}"""
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0.3,
        system="Summarize academic papers clearly.",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

def answer_question(text, question):
    prompt = f"""You are an expert academic assistant.
Answer the following question based on the content of this research paper:

Paper Content:
{text[:12000]}

Question: {question}

Answer:"""
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=500,
        temperature=0.2,
        system="You answer academic questions clearly and concisely.",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

def generate_glossary(text):
    prompt = f"""You are an AI assistant trained to simplify academic language.

From the following paper, extract the 5–10 most important technical terms or concepts. For each term, write a short definition in simple, clear language.

Output in this format:

Term: Definition

Text:
{text[:12000]}"""
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=800,
        temperature=0.3,
        system="You generate glossaries from technical papers.",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

def explain_like_12(text):
    prompt = f"""You are an educational AI assistant.

Explain the following research paper as if you're speaking to a curious 12-year-old. Avoid jargon and keep it simple, like you're telling a story.

Text:
{text[:12000]}"""
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=800,
        temperature=0.4,
        system="You explain complex topics in a child-friendly way.",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

# --------------------- UI --------------------- #

st.title("📘 Claude-Powered Paper Summarizer")
uploaded_file = st.file_uploader("Upload a research paper (PDF)", type=["pdf"])

if uploaded_file:
    with st.spinner("Extracting text..."):
        paper_text = extract_text_from_pdf(uploaded_file)
    st.success("Text extracted!")

    # --- Control Buttons ---
    col1, col2, col3 = st.columns(3)
    with col1:
        if st.button("🧠 Summarize"):
            st.session_state.generate_summary = True
            st.session_state.ask_question = False
            st.session_state.generate_glossary = False
            st.session_state.simplify_text = False
    with col2:
        if st.button("📘 Glossary"):
            st.session_state.generate_summary = False
            st.session_state.ask_question = False
            st.session_state.generate_glossary = True
            st.session_state.simplify_text = False
    with col3:
        if st.button("🧒 Explain Simply"):
            st.session_state.generate_summary = False
            st.session_state.ask_question = False
            st.session_state.generate_glossary = False
            st.session_state.simplify_text = True

    # --- Q&A Input ---
    st.subheader("🧠 Ask a question about this paper:")
    question_text = st.text_input(
        "Type your question and click Ask",
        placeholder="e.g. What are the key findings in 2 sentences?"
    )

    if st.button("🔍 Ask") and question_text.strip():
        st.session_state.ask_question = True
        st.session_state.generate_summary = False
        st.session_state.generate_glossary = False
        st.session_state.simplify_text = False
        st.session_state["question_to_answer"] = question_text

    # --- Run Selected Feature ---

    if st.session_state.generate_summary:
        with st.spinner("Generating summary..."):
            summary = summarize_with_claude(paper_text)
        st.subheader("📝 Summary")
        st.write(summary)
        st.session_state.generate_summary = False

    if st.session_state.ask_question:
        with st.spinner("Answering your question..."):
            answer = answer_question(paper_text, st.session_state["question_to_answer"])
        st.subheader("💬 Answer")
        st.write(answer)
        st.session_state.ask_question = False
        del st.session_state["question_to_answer"]

    if st.session_state.generate_glossary:
        with st.spinner("Extracting glossary..."):
            glossary = generate_glossary(paper_text)
        st.subheader("📘 Glossary of Key Terms")
        st.text(glossary)
        st.session_state.generate_glossary = False

    if st.session_state.simplify_text:
        with st.spinner("Simplifying the paper..."):
            simple_explanation = explain_like_12(paper_text)
        st.subheader("🧒 Simplified Explanation")
        st.write(simple_explanation)
        st.session_state.simplify_text = False


Overwriting app.py


In [25]:
# Start Streamlit in background
!streamlit run app.py &>/content/logs.txt &

# Now create the tunnel using HTTP
public_url = ngrok.connect("8501", "http")  # explicitly specify HTTP type
print("🌐 Streamlit Public URL:", public_url)


🌐 Streamlit Public URL: NgrokTunnel: "https://6280-34-82-124-236.ngrok-free.app" -> "http://localhost:8501"
